In [1]:
#インストール
#!pip install selenium
#!pip install chromedriver-binary-auto
#!pip install webdriver-manager

In [2]:
#スクレイピング
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup


from selenium import webdriver
from selenium.webdriver.support.ui import Select,WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

import os
import time
import requests

In [3]:
driver = webdriver.Chrome(ChromeDriverManager().install())
wait = WebDriverWait(driver,10)

URL = "https://db.netkeiba.com/?pid=horse_search_detail"
driver.get(URL)
time.sleep(1)
wait.until(EC.presence_of_all_elements_located)

C:\Users\kawam\AppData\Local\Temp\ipykernel_30948\2276666827.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


<function selenium.webdriver.support.expected_conditions.presence_of_all_elements_located.<locals>._predicate(driver)>

In [4]:
age = 'none'
color = 'chestnut-hair'
division = 'none'

age_element =driver.find_element(By.NAME,"under_age")
age_select = Select(age_element)
age_select.select_by_value(str(age))

In [5]:
#color 1から
for i in range(1,2):
    terms = driver.find_element(By.ID,"check_color_0"+ str(i))
    terms.click()

In [6]:
#division 0から
for i in range(0,1):
    terms = driver.find_element(By.ID,"check_umamark_"+ str(i))
    terms.click()

In [7]:
list_element = driver.find_element(By.NAME,'list')
list_select = Select(list_element)
list_select.select_by_value("100")

In [8]:
# フォームを送信
frm = driver.find_element(By.CSS_SELECTOR,"#db_search_detail_form > form")
frm.submit()
time.sleep(5)
wait.until(EC.presence_of_all_elements_located)

<function selenium.webdriver.support.expected_conditions.presence_of_all_elements_located.<locals>._predicate(driver)>

In [9]:
if not os.path.exists('text'):
    os.makedirs('text')

with open("text/"+str(color)+"-"+str(division)+".txt", mode='w') as f:
    while True:
        time.sleep(5)
        wait.until(EC.presence_of_all_elements_located)
        all_rows = driver.find_element(By.CLASS_NAME,'race_table_01').find_elements(By.TAG_NAME,"tr")
        for row in range(1, len(all_rows)):
            race_href=all_rows[row].find_elements(By.TAG_NAME,"td")[1].find_element(By.TAG_NAME,"a").get_attribute("href")
            f.write(race_href+"\n")
        try:
            target = driver.find_elements(By.LINK_TEXT,"次")[0]
            driver.execute_script("arguments[0].click();", target) 
        except IndexError:
            break

In [10]:
save_dir = "html"+"/"+str(color)+"-"+str(division)
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
        
with open("text/"+str(color)+"-"+str(division)+".txt", "r") as f:
    urls = f.read().splitlines()
    for url in urls:
        list = url.split("/")
        horse_id = list[-2]
        save_file_path = save_dir+"/"+horse_id+'.html'
        response = requests.get(url)
        response.encoding = response.apparent_encoding
        html = response.text
        time.sleep(5)
        with open(save_file_path, 'w') as file:
            file.write(html)

UnicodeEncodeError: 'cp932' codec can't encode character '\ufffd' in position 38675: illegal multibyte sequence

In [ ]:
horse_data_info_columns=[
    'horse_id',
    'bday',
    'tame_id',
    'owner_id',
    'producer',
    'production area',
    'auction price',
    'winnings',
    'lifetime record',
    'wined race title',
    'inbreeding-1',
    'inbreeding-2',
    'father',
    'faths father',
    'faths mother',
    'mother',
    'moths father',
    'moths mother'
]

In [ ]:
def get_horse_data_by_html(horse_id, html):
    horse_list = [horse_id]
    soup = BeautifulSoup(html, 'html.parser')
    horse_tables = soup.find("table", class_="db_prof_table")
    horse_table = horse_tables.findAll("td")
    #for i in range(10):
    horse_list.append(horse_table[0].get_text())
    horse_list.append(horse_table[1].find('a').get('href').split("/")[-2])
    horse_list.append(horse_table[2].find('a').get('href').split("/")[-2])
    horse_list.append(horse_table[3].find('a').get('href').split("/")[-2])
    horse_list.append(horse_table[4].get_text())
    horse_list.append(horse_table[5].get_text())
    horse_list.append(horse_table[6].get_text())
    horse_list.append(horse_table[7].get_text())
    horse_list.append(horse_table[8].find('a').get('href').split("/")[-2])
    
    # horse_table[8]からすべてのリンクのURLの一部を取得する
    url_parts = [link.get('href').split("/")[-2] for link in horse_table[9].find_all('a')]

    # url_partsの要素数が2未満の場合、残りの要素をNaNで埋める
    while len(url_parts) < 2:
        url_parts.append(np.nan)

    # horse_listにurl_partsを追加する
    horse_list.extend(url_parts)
    
    #horse_list.append(horse_table[9].get_text())
    blood_tables = soup.find("table", class_="blood_table")
    blood_table = blood_tables.findAll("td")
    #for i in range(6):
    horse_list.append(blood_table[0].find('a').get('href').split("/")[-2])
    horse_list.append(blood_table[1].find('a').get('href').split("/")[-2])
    horse_list.append(blood_table[2].find('a').get('href').split("/")[-2])
    horse_list.append(blood_table[3].find('a').get('href').split("/")[-2])
    horse_list.append(blood_table[4].find('a').get('href').split("/")[-2])
    horse_list.append(blood_table[5].find('a').get('href').split("/")[-2])
    return horse_list

In [ ]:
CSV_DIR = "csv"
if not os.path.isdir(CSV_DIR):
    os.makedirs(CSV_DIR)
horse_info_csv = CSV_DIR+"/horse-info-"+str(color)+"-"+str(division)+".csv"

horse_df = pd.DataFrame(columns=horse_data_info_columns)

html_dir = "html"+"/"+str(color)+"-"+str(division)
if os.path.isdir(html_dir):
    file_list = os.listdir(html_dir)
    for file_name in file_list:
        with open(html_dir+"/"+file_name, "r") as f:
            html = f.read()
            list = file_name.split(".")
            horse_id = list[-2]
            horse_list = get_horse_data_by_html(horse_id, html) 
            horse_se = pd.Series(horse_list, index=horse_df.columns)
            horse_df = pd.concat([horse_df, horse_se.to_frame().T], ignore_index=True)

In [ ]:
horse_df.to_csv(horse_race_csv, header=True, index=False)

In [ ]:
horse_df